# Le notebook que yohan n'a pas fait

In [1]:
import pandas as pd
from matplotlib import pyplot as plt
import warnings
warnings.filterwarnings('ignore')
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
import os

In [2]:
caracteristiques = pd.read_csv('./Dataset/all_years/caracteristiques-2019-22.csv')
departements = pd.read_csv('./Dataset/departements-france.csv')
lieux = pd.read_csv('./Dataset/all_years/lieux-2019-22.csv')
usager = pd.read_csv('./Dataset/all_years/usagers-2019-22.csv')
vehicule = pd.read_csv('./Dataset/all_years/vehicules-2019-22.csv')

In [3]:
def merge_vehicle_count(caracteristiques, vehicules):
    # Compter le nombre de véhicules par "Num_Acc" dans le DataFrame vehicules
    vehicules_count = vehicules.groupby("Accident_Id")["id_vehicule"].count().reset_index().rename(columns={'id_vehicule':'Nombre_vehicule'})
    
    # Fusionner le DataFrame caracteristiques avec le DataFrame vehicules_count en utilisant la colonne "Num_Acc"
    merged_df = caracteristiques.merge(vehicules_count, on='Accident_Id')
    
    return merged_df


def merge_user_count(caracteristiques,usagers):
    # Compter le nombre d'usagers par "Accident_Id" et par sexe dans le DataFrame usagers
    usagers_count = usagers.groupby(["Accident_Id", "sexe"])["id_vehicule"].count().unstack(fill_value=0)
    usagers_count.rename(columns={1: 'Nombre_Hommes', 2: 'Nombre_Femmes'}, inplace=True)
    
    # Fusionner le DataFrame caracteristiques avec usagers_count en utilisant la colonne "Num_Acc"
    merged_df = caracteristiques.merge(usagers_count, on='Accident_Id')
    
    return merged_df

def replace_with_mapper(df, col, mapper):
    df[col] = df[col].replace(mapper)
    
    return df

# Remplacement des heures par le moment de la journée
matinee = ['06:','07:','08:','09:','10:']
journee = ['11:','12:','13:','14:','15:','16:','17:']
soiree = ['18:','19:','20:','21:','22:','23:']
nuit = ['00:','01:','02:','03:','04:','05:']

def replace_hour(hour):
    hour = hour[:3]
    if hour in matinee:
        return 'Matinée'
    elif hour in journee:
        return 'Journée'
    elif hour in soiree:
        return 'Soirée'
    elif hour in nuit:
        return 'Nuit'
    else:
        'Lucas'

In [4]:
years = ['19', '20', '21', '22']
files_name = ["caracteristiques-20", "lieux-20", "usagers-20", "vehicules-20"]
if not os.path.exists('./Dataset/all_years'):
    os.mkdir('./Dataset/all_years/', 0o666) 
for file_name in files_name:
    result = []
    for year in years:
        file_path = f'./Dataset/{year}/{file_name}{year}.csv'
        sep = ';'
        df = pd.read_csv(file_path, sep=sep, low_memory=False)
        # if 'id_usager' in df.columns:
        #     df = df.drop(columns=['id_usager'])
        result.append(df)
    combined_df = pd.concat(result, ignore_index=True)
    combined_df.to_csv(f'./Dataset/all_years/{file_name}{years[0]}-{years[-1]}.csv', index=False)

In [5]:
caracteristiques = caracteristiques.drop(['com', 'adr', 'lat', 'long'], axis=1)

# Remplacer les numéros des mois par leurs noms
caracteristiques.mois = caracteristiques.mois.map({
    1:'Janvier',
    2:'Février',
    3:'Mars',
    4:'Avril',
    5:'Mai',
    6:'Juin',
    7:'Juillet',
    8:'Août',
    9:'Septembre',
    10:'Octobre',
    11:'Novembre',
    12:'Décembre'
}, na_action=None)

caracteristiques.hrmn = caracteristiques.hrmn.apply(replace_hour)

caracteristiques = replace_with_mapper(caracteristiques, 'lum', {
    1 : 'Plein jour',
    2 : 'Crépuscule ou aube',
    3 : 'Nuit sans éclairage public',
    4 : 'Nuit avec éclairage public non allumé',
    5 : 'Nuit avec éclairage public allumé'
})

# Remplacer les numéros de départements par leur noms
departements_mapper = departements.set_index('code_departement')['nom_departement'].to_dict()
caracteristiques = replace_with_mapper(caracteristiques, 'dep', departements_mapper)

caracteristiques = replace_with_mapper(caracteristiques, 'agg', {
    1 : 'Hors agglomération',
    2 : 'En agglomération'
})
caracteristiques = replace_with_mapper(caracteristiques, 'int', {
    1 : 'Hors intersection',
    2 : 'Intersection en X',
    3 : 'Intersection en T',
    4 : 'Intersection en Y',
    5 : 'Intersection à plus de 4 branches',
    6 : 'Giratoire',
    7 : 'Place',
    8 : 'Passage à niveau',
    9 : 'Autre intersection'
})
caracteristiques = replace_with_mapper(caracteristiques, 'atm', {
    -1 :'Non renseigné',
    1 : 'Normale',
    2 : 'Pluie légère',
    3 : 'Pluie forte',
    4 : 'Neige - grêle',
    5 : 'Brouillard - fumée',
    6 : 'Vent fort - tempête',
    7 : 'Temps éblouissant',
    8 : 'Temps couvert',
    9 : 'Autre'
})
caracteristiques = replace_with_mapper(caracteristiques, 'col', {
    -1 :'Non renseigné',
    1 : 'Deux véhicules - frontale',
    2 : 'Deux véhicules – par l’arrière',
    3 : 'Deux véhicules – par le coté',
    4 : 'Trois véhicules et plus – en chaîne',
    5 : 'Trois véhicules et plus - collisions multiples',
    6 : 'Autre collision',
    7 : 'Sans collision'
})

caracteristiques = caracteristiques.merge(
replace_with_mapper(lieux.loc[:, ['Accident_Id', 'catr']], 'catr', {
    1:'Autoroute',
    2:'Route nationale',
    3:'Route Départementale',
    4:'Voie Communales',
    5:'Hors réseau public',
    6:'Parc de stationnementouvert à la circulation publique',
    7:'Routes de métropole urbaine',
    9:'autre'
}), on='Accident_Id', how='left')

caracteristiques = caracteristiques.merge(
replace_with_mapper(lieux.loc[:, ['Accident_Id', 'circ']], 'circ', {
    -1:'Non renseigné',
    1:'A sens unique',
    2:'Bidirectionnelle',
    3:'A chaussées séparées',
    4:'Avec voies d’affectation variable'
}), on='Accident_Id', how='left')

caracteristiques = caracteristiques.merge(lieux.loc[:, ['Accident_Id', 'nbv']], on='Accident_Id', how='left')

caracteristiques = caracteristiques.merge(
replace_with_mapper(lieux.loc[:, ['Accident_Id', 'surf']], 'surf', {
    -1:'Non renseigné',
    1:'Normale',
    2:'Mouillée',
    3:'Flaques',
    4:'Inondée',
    5:'Enneigée',
    6:'Boue',
    7:'Verglacée',
    8:'Corps gras – huile',
    9:'Autre'
}), on='Accident_Id', how='left')

caracteristiques = caracteristiques.merge(
replace_with_mapper(lieux.loc[:, ['Accident_Id', 'infra']], 'infra', {
    -1:'Non renseigné',
    0:'Aucun',
    1:'Souterrain -tunnel',
    2:'Pont -autopont',
    3:'Bretelle d’échangeur ou de raccordement',
    4:'Voie ferrée',
    5:'Carrefour aménagé',
    6:'Zone piétonne',
    7:'Zone de péage',
    8:'Chantier',
    9:'Autres'
}), on='Accident_Id', how='left')

caracteristiques = caracteristiques.merge(
replace_with_mapper(lieux.loc[:, ['Accident_Id', 'situ']], 'situ', {
    -1:'Non renseigné',
    0:'Aucun',
    1:'Sur chaussée',
    2:'Sur bande d’arrêt d’urgence',
    3:'Sur accotement',
    4:'Sur trottoir',
    5:'Sur piste cyclable',
    6:'Sur autre voie spéciale',
    8:'Autres'
}), on='Accident_Id', how='left')

caracteristiques = caracteristiques.merge(lieux.loc[:, ['Accident_Id', 'vma']], on='Accident_Id', how='left')

caracteristiques.rename(columns={
    'an': 'année',
	'hrmn': 'horraire',
    'lum': 'luminosité',
    'dep': 'département',
    'agg': 'agglomération',
    'int': 'intersection',
    'atm': 'atmosphere',
    'col': 'Collision',
    'catr': 'catégorie de route',
    'circ': 'régime de circulation',
    'nbv': 'nombre de voix',
    'surf': 'etat de la surface',
    'infra': 'infrastructure',
    'situ': 'situation',
    'vma': 'vitesse max autorisée'
}, inplace=True)

caracteristiques.head(5)

,Accident_Id,jour,mois,année,horraire,luminosité,département,agglomération,intersection,atmosphere,Collision,catégorie de route,régime de circulation,nombre de voix,etat de la surface,infrastructure,situation,vitesse max autorisée
0,201900000001,30,Novembre,2019,Nuit,Nuit avec éclairage public non allumé,Seine-Saint-Denis,Hors agglomération,Hors intersection,Normale,Deux véhicules – par l’arrière,Autoroute,A chaussées séparées,10,Normale,Pont -autopont,Sur chaussée,70
1,201900000002,30,Novembre,2019,Nuit,Nuit sans éclairage public,Seine-Saint-Denis,Hors agglomération,Hors intersection,Normale,Autre collision,Autoroute,A sens unique,2,Normale,Aucun,Sur chaussée,70
2,201900000003,28,Novembre,2019,Journée,Plein jour,Hauts-de-Seine,Hors agglomération,Hors intersection,Normale,Trois véhicules et plus – en chaîne,Autoroute,A chaussées séparées,8,Normale,Aucun,Sur chaussée,90
3,201900000004,30,Novembre,2019,Soirée,Nuit avec éclairage public allumé,Val-de-Marne,Hors agglomération,Hors intersection,Normale,Trois véhicules et plus – en chaîne,Autoroute,A chaussées séparées,5,Normale,Aucun,Sur chaussée,90
4,201900000005,30,Novembre,2019,Nuit,Nuit sans éclairage public,Val-de-Marne,Hors agglomération,Hors intersection,Normale,Deux véhicules – par l’arrière,Autoroute,A sens unique,3,Normale,Pont -autopont,Sur chaussée,90


In [6]:
r = usager.groupby('Accident_Id')['catu'].value_counts().unstack().fillna(0)
r.reset_index(inplace=True)
r.rename(columns={
    1:'Conducteur',
    2:'Passager',
    3:'Piéton'
}, inplace=True)
caracteristiques = caracteristiques.merge(r, on='Accident_Id', how='left')

r = usager.groupby('Accident_Id')['grav'].value_counts().unstack().fillna(0)
r.reset_index(inplace=True)
r.rename(columns={
    -1:'gravité indéterminé',
    1:'Indemne',
    2:'Tué',
    3:'Blessé hospitalisé',
    4:'Blessé léger'
}, inplace=True)
caracteristiques = caracteristiques.merge(r, on='Accident_Id', how='left')

r = usager.groupby('Accident_Id')['sexe'].value_counts().unstack().fillna(0)
r.reset_index(inplace=True)
r.rename(columns={
    -1:'Autre',
    1:'Hommes',
    2:'Femmes'
}, inplace=True)
caracteristiques = caracteristiques.merge(r, on='Accident_Id', how='left')

r = vehicule.groupby('Accident_Id')['catv'].value_counts().unstack().fillna(0)
r.reset_index(inplace=True)
r.rename(columns={
    -1:'Autre',
    1:'Hommes',
    2:'Femmes'
}, inplace=True)
caracteristiques = caracteristiques.merge(r, on='Accident_Id', how='left')

caracteristiques.head(5)

NameError: name 've' is not defined